# OilyGiant

## Visão do Projeto

Você trabalha na empresa de mineração OilyGiant. Sua tarefa é encontrar os melhores lugares para o **desenvolvimento de novos poços de petróleo**.

Para concluir essa tarefa, você vai precisar executar as seguintes etapas:

- Ler os arquivos com parâmetros coletados de poços de petróleo na região selecionada: a quantidade de petróleo e o volume de reservas;

- Construir um modelo para predizer o volume de reservas em novos poços;

- Escolher os poços de petróleo que têm os maiores valores estimados;

- Escolher a região com o maior lucro total para os poços de petróleo selecionados.

Você tem dados sobre amostras de petróleo de três regiões. Os parâmetros de cada poço de petróleo na região já são conhecidos. Construa um modelo que te ajudará a escolher a região com a margem de lucro mais alta. Utilize a técnica de Bootstrapping para analisar lucro potencial e riscos.

---

# 1. INICIALIZAÇÃO

# 1.1 Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Impedir avisos
import warnings
warnings.filterwarnings("ignore")

# Divisào do dataset
from sklearn.model_selection import train_test_split

# Modelo
from sklearn.linear_model import LinearRegression

# Metricas
from sklearn.metrics import mean_squared_error

# Pipeline
from sklearn.pipeline import Pipeline

# Column Transformer
from sklearn.compose import ColumnTransformer

# Pre processamento
from sklearn.preprocessing import StandardScaler

## 1.2 Carregando Dataset

In [2]:
# Carregue os arquivos de dados em diferentes DataFrames

# Local Path
local_geo_0_path = "geo_data_0.csv"
local_geo_1_path = "geo_data_1.csv"
local_geo_2_path = "geo_data_2.csv"

# Cloud Path
geo_0_path = "/datasets/geo_data_0.csv"
geo_1_path = "/datasets/geo_data_1.csv"
geo_2_path = "/datasets/geo_data_2.csv"

try:
    geo_0 = pd.read_csv(local_geo_0_path, sep=",")
    geo_1 = pd.read_csv(local_geo_1_path, sep=",")
    geo_2 = pd.read_csv(local_geo_2_path, sep=",")

except FileNotFoundError:
    print(
        f"O arquivo CSV não foi encontrado em {local_geo_0_path}. Tentando o caminho {geo_0_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_geo_1_path}. Tentando o caminho {geo_1_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_geo_2_path}. Tentando o caminho {geo_2_path}..."
    )

    try:
        geo_0 = pd.read_csv(geo_0_path, sep=",")
        geo_1 = pd.read_csv(geo_1_path, sep=",")
        geo_2 = pd.read_csv(geo_2_path, sep=",")

    except FileNotFoundError:
        print(
            f"O arquivo CSV não foi encontrado em {geo_0_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {geo_1_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {geo_2_path}. Nenhum arquivo encontrado."
        )

In [3]:
# Checando se DataFrame foi importado
geo_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# Checando se DataFrame foi importado
geo_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
# Checando se DataFrame foi importado
geo_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## 1.3 Analisando os Dados

#### 1.3.1 Dataset: geo_0

In [6]:
# dtypes dos dados
geo_0.dtypes

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [7]:
# Info dos dados
geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
# Checando dados faltosos
geo_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [9]:
# Checando duplicados
geo_0.duplicated().sum()

0

In [10]:
# Describe dos dados
geo_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


- O dataset geo_0 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

#### 1.3.2 Dataset: geo_1

In [11]:
# dtypes dos dados
geo_1.dtypes

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [12]:
# Info dos dados
geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
# Checando dados faltosos
geo_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [14]:
# Checando duplicados
geo_1.duplicated().sum()

0

In [15]:
# Describe dos dados
geo_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


- O dataset geo_1 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

#### 1.3.3 Dataset: geo_2

In [16]:
# dtypes dos dados
geo_2.dtypes

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [17]:
# Info dos dados
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [18]:
# Checando dados faltosos
geo_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [19]:
# Checando duplicados
geo_2.duplicated().sum()

0

In [20]:
# Describe dos dados
geo_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


- O dataset geo_2 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

## 2. PREPARAÇÃO DO MODELO

In [21]:
def preprocess_train_model(df):
    # Separando o dataset em traino e validação
    train_data, valid_data = train_test_split(
        df, test_size=0.25, random_state=12345)

    # Definir as features e target
    features_train, target_train = (
        train_data.drop(["product"], axis=1),
        train_data["product"],
    )
    features_valid, target_valid = (
        valid_data.drop(["product"], axis=1),
        valid_data["product"],
    )

    # Criando pipeline de pre-processamento.
    numeric_features = ["f0", "f1", "f2"]
    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    preprocessor = ColumnTransformer(
        transformers=[("num", numeric_transformer, numeric_features)]
    )

    model = Pipeline(
        steps=[("preprocessor", preprocessor),
               ("regressor", LinearRegression())]
    )

    # Treinar o modelo
    model.fit(features_train, target_train)

    # Predições no conjunto de validação
    predictions = model.predict(features_valid)

    # Salvar predições
    results_df = pd.DataFrame(
        {"true_values": target_valid, "predicted": predictions})

    # Métricas de avaliação
    avg_predicted_volume = predictions.mean()
    rmse = mean_squared_error(target_valid, predictions, squared=False)

    print(f"Volume Médio Previsto: {avg_predicted_volume:.4f}")
    print(f"REQM: {rmse:.4f}")

    return results_df

In [22]:
# Rodando o modelo para o dataset: geo_0
results_0 = preprocess_train_model(geo_0)
results_0

Volume Médio Previsto: 92.5926
REQM: 37.5794


,true_values,predicted
71751,10.038645,95.894952
80493,114.551489,77.572583
2655,132.603635,77.892640
53233,169.072125,90.175134
91141,122.325180,70.510088
...,...,...
12581,170.116726,103.037104
18456,93.632175,85.403255
73035,127.352259,61.509833
63834,99.782700,118.180397


- O volume médio de produção para a região 0 é de 92.5926 milhares de barris

- Podemos ver que para o conjunto de dados de geo_0, temos um Root Mean Squared Error (REQM) igual à 37,5794. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 37,5794 milhares de barris em relação aos valores reais.

Lembrando que quanto menor o valor do REQM, melhor o desempenho do modelo. Nesse caso, seria desejável ter um REQM mais baixo para indicar uma precisão maior nas predições do modelo de regressão linear.

In [23]:
# Rodando o modelo para o dataset: geo_1
results_1 = preprocess_train_model(geo_1)
results_1

Volume Médio Previsto: 68.7285
REQM: 0.8931


,true_values,predicted
71751,80.859783,82.663314
80493,53.906522,54.431786
2655,30.132364,29.748760
53233,53.906522,53.552133
91141,0.000000,1.243856
...,...,...
12581,137.945408,136.869211
18456,110.992147,110.693465
73035,137.945408,137.879341
63834,84.038886,83.761966


- O volume médio de produção para a região 1 é de 68.7285 milhares de barris

- Podemos ver que para o conjunto de dados de geo_01 temos um Root Mean Squared Error (REQM) igual à 0,8931. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 0,8931 milhares de barris em relação aos valores reais.

Ou seja, o modelo prevê com muita precisão o volume de reservas no poço de petróleo.

In [24]:
# Rodando o modelo para o dataset: geo_2
results_2 = preprocess_train_model(geo_2)
results_2

Volume Médio Previsto: 94.9650
REQM: 40.0297


,true_values,predicted
71751,61.212375,93.599633
80493,41.850118,75.105159
2655,57.776581,90.066809
53233,100.053761,105.162375
91141,109.897122,115.303310
...,...,...
12581,28.492402,78.765887
18456,21.431303,95.603394
73035,125.487229,99.407281
63834,99.422903,77.779912


- O volume médio de produção para a região 2 é de 94.9650 milhares de barris

- Podemos ver que para o conjunto de dados de geo_2, temos um Root Mean Squared Error (REQM) igual à 40,0297. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 40,0297 milhares de barris em relação aos valores reais.

Dentre os 3 conjuntos de dados (geo_0, geo_1 e geo_2), esse conjunto teve o maior desvio, em média, dos valores reais. 

## 3. CALCULANDO O LUCRO

In [25]:
# Valores necessários para calculo
investment = 100000000
revenue_per_unit = 4500
required_units = 111.1

# Calculo do volume médio necessário para evitar prejuízos
volume_needed = investment / (revenue_per_unit * required_units)

In [26]:
# Valor médio de cada região
avg_volume_0 = geo_0["product"].mean()
avg_volume_1 = geo_1["product"].mean()
avg_volume_2 = geo_2["product"].mean()

In [27]:
print(f"Volume Médio Necessário: {volume_needed:.4f}")
print(f"Volume Médio da Região 0: {avg_volume_0:.4f}")
print(f"Volume Médio da Região 1: {avg_volume_1:.4f}")
print(f"Volume Médio da Região 2: {avg_volume_2:.4f}")

Volume Médio Necessário: 200.0200
Volume Médio da Região 0: 92.5000
Volume Médio da Região 1: 68.8250
Volume Médio da Região 2: 95.0000


A região que tem o maior volume de produção é a região 2, seguida da região 0 e por fim a região 1.

## 4. FUNÇÃO PARA CALCULAR O LUCRO DE UM CONJUNTO DE DADOS

In [28]:
def calculate_profit(selected_wells, predictions):
    # Selecionando os 200 poços com os valores mais altos previstos
    selected_wells = predictions.nlargest(200, "predicted")

    # Soma do volume de reservas
    target_volume = selected_wells["true_values"].sum()

    # Calculo do lucro
    profit = target_volume * revenue_per_unit - investment

    return profit

In [29]:
# Calcular lucro potencial para região 0
profit_0 = calculate_profit(geo_0, results_0)
profit_0

33208260.43139851

In [30]:
# Calcular lucro potencial para região 1
profit_1 = calculate_profit(geo_1, results_1)
profit_1

24150866.966815114

In [31]:
# Calcular lucro potencial para região 2
profit_2 = calculate_profit(geo_2, results_2)
profit_2

27103499.635998324

In [32]:
# Apresentar conclusões sobre lucro potencial
print(f"Lucro Potencial Região 0: {profit_0:.2f}")
print(f"Lucro Potencial Região 1: {profit_1:.2f}")
print(f"Lucro Potencial Região 2: {profit_2:.2f}")

Lucro Potencial Região 0: 33208260.43
Lucro Potencial Região 1: 24150866.97
Lucro Potencial Região 2: 27103499.64


- Região 0: O lucro potencial estimado para a região 0 é de aproximadamente 33.2 milhões.
- Região 1: Para a região 1, o lucro potencial é cerca de 24.2 milhões.
- Região 2: O lucro potencial estimado para a região 2 é em torno de 27.1 milhões.

### 5. CALCULANDO RISCOS E LUCROS PARA CADA REGIÃO

In [33]:
def calculate_bootstrap(data, predictions, iterations=1000):
    state = np.random.RandomState(12345)
    profits = []

    for i in range(iterations):
        # Amostrar com reposição
        sampled_size = min(len(data), len(predictions))
        sampled_data = data.sample(
            n=sampled_size, replace=True, random_state=state)

        # Verificar se os tamanhos dos DataFrames são iguais
        if len(sampled_data) == len(predictions):
            # Usar índices numéricos com iloc
            sampled_data.reset_index(drop=True, inplace=True)
            sampled_predictions = predictions.reset_index(drop=True).iloc[
                sampled_data.index
            ]

            # Calcular lucro potencial para a amostra
            profit = calculate_profit(sampled_data, sampled_predictions)
            profits.append(profit)
        else:
            print("Tamanhos diferentes entre sampled_data e predictions.")

    mean_profit = np.mean(profits)
    ci_lower, ci_upper = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = np.mean(np.array(profits) < 0) * 100

    return mean_profit, ci_lower, ci_upper, risk_of_loss

In [34]:
# Calcular para cada região
bootstrap_results_0 = calculate_bootstrap(geo_0, results_0)
bootstrap_results_1 = calculate_bootstrap(geo_1, results_1)
bootstrap_results_2 = calculate_bootstrap(geo_2, results_2)

In [35]:
# Apresentar conclusões sobre riscos e lucro
print("Região 0:")
print(f"Lucro Médio: {bootstrap_results_0[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_0[1]:.2f}, {bootstrap_results_0[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_0[3]:.2f}%\n\n")

print("Região 1:")
print(f"Lucro Médio: {bootstrap_results_1[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_1[1]:.2f}, {bootstrap_results_1[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_1[3]:.2f}%\n\n")

print("Região 2:")
print(f"Lucro Médio: {bootstrap_results_2[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_2[1]:.2f}, {bootstrap_results_2[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_2[3]:.2f}%")

Região 0:
Lucro Médio: 33208260.43
Intervalo de Confiança (95%): (33208260.43, 33208260.43)
Risco de Prejuízo: 0.00%


Região 1:
Lucro Médio: 24150866.97
Intervalo de Confiança (95%): (24150866.97, 24150866.97)
Risco de Prejuízo: 0.00%


Região 2:
Lucro Médio: 27103499.64
Intervalo de Confiança (95%): (27103499.64, 27103499.64)
Risco de Prejuízo: 0.00%


Com base nos resultados obtidos para cada região usando a técnica de bootstrap, podemos fazer as seguintes conclusões:

- Lucro Médio:

   - Região 0: O lucro médio é de aproximadamente 33.2 milhões.
   - Região 1: A média do lucro é cerca de 24.2 milhões.
   - Região 2: O lucro médio é em torno de 27.1 milhões.

- Intervalo de Confiança (95%):

    - Para todas as regiões, o intervalo de confiança (95%) é bastante restrito, indicando uma precisão considerável nos resultados do bootstrap. Isso significa que, com 95% de confiança, espera-se que o verdadeiro valor do lucro esteja dentro do intervalo fornecido.

- Risco de Prejuízo:

Para todas as regiões, o risco de prejuízo é relatado como 0.00%. Isso indica que, com base nas amostras bootstrap, não foram observados cenários em que o lucro foi negativo.

## 6. CONCLUSÃO

Com base nos cálculos de lucro potencial para cada região, podemos tirar as seguintes conclusões:

- Lucro Potencial:

    - Região 0: O lucro potencial estimado para a região 0 é de aproximadamente 33.2 milhões.
    - Região 1: Para a região 1, o lucro potencial é cerca de 24.2 milhões.
    - Região 2: O lucro potencial estimado para a região 2 é em torno de 27.1 milhões.

- Comparação com os Resultados do Bootstrap:

    Os valores de lucro potencial obtidos diretamente do cálculo para cada região são consistentes com os resultados obtidos através da técnica de bootstrap. Ambas as abordagens indicam desempenho positivo em termos de lucro.

- Ranking de Regiões em Termos de Lucro Potencial:

    - A região 0 apresenta o maior lucro potencial entre as três regiões.
    - A região 2 possui um lucro potencial ligeiramente superior à região 1.

Portanto, com base nos cálculos de lucro potencial, a região 0 é a mais promissora em termos financeiros, seguida pela região 2 e, por fim, a região 1. Essas conclusões são fundamentadas na suposição de que os cálculos de lucro potencial são representativos do desempenho real das regiões em relação aos poços de petróleo.